In [36]:
import sys
sys.path.append("../data/")
sys.path.append("../misc/")
sys.path.append("../models/")
sys.path.append("../preprocessing/")
sys.path.append("../tools/")


import json
import random
import numpy as np
from PIL import Image
from plot_utils import PlotMaps, PlotHeatmaps

In [37]:
data_dir = f'../data/HuPR/single_{random.randint(1, 10)}'

hori_bin = data_dir + '/hori/adc_data.bin'
vert_bin = data_dir + '/vert/adc_data.bin'

# variables
numLanes = 2
numRX = 4
numADCSamples = 256
adcRatio = 4
numChirp = 64 * 3
framePerSecond = 10
duration = 60
numFrame = framePerSecond * duration

# open the binary file
def process_data(bin_file, num_lanes, numRX, numADCSamples, adcRatio):
    adcData = np.fromfile(bin_file, dtype=np.int16)
    fileSize = adcData.shape[0]
    adcData = adcData.reshape(-1, numLanes*2).transpose()
    # # for complex data
    fileSize = int(fileSize/2)
    LVDS = np.zeros((2, fileSize))  # seperate each LVDS lane into rows

    temp = np.empty((adcData[0].size + adcData[1].size), dtype=adcData[0].dtype)
    temp[0::2] = adcData[0]
    temp[1::2] = adcData[1]
    LVDS[0] = temp
    temp = np.empty((adcData[2].size + adcData[3].size), dtype=adcData[2].dtype)
    temp[0::2] = adcData[2]
    temp[1::2] = adcData[3]
    LVDS[1] = temp

    adcData = np.zeros((numRX, int(fileSize/numRX)), dtype = 'complex_')
    iter = 0
    for i in range(0, fileSize, numADCSamples * 4):
        adcData[0][iter:iter+numADCSamples] = LVDS[0][i:i+numADCSamples] + np.sqrt(-1+0j)*LVDS[1][i:i+numADCSamples]
        adcData[1][iter:iter+numADCSamples] = LVDS[0][i+numADCSamples:i+numADCSamples*2] + np.sqrt(-1+0j)*LVDS[1][i+numADCSamples:i+numADCSamples*2]
        adcData[2][iter:iter+numADCSamples] = LVDS[0][i+numADCSamples*2:i+numADCSamples*3] + np.sqrt(-1+0j)*LVDS[1][i+numADCSamples*2:i+numADCSamples*3]
        adcData[3][iter:iter+numADCSamples] = LVDS[0][i+numADCSamples*3:i+numADCSamples*4] + np.sqrt(-1+0j)*LVDS[1][i+numADCSamples*3:i+numADCSamples*4]
        iter = iter + numADCSamples
    
    #correct reshape
    adcDataReshape = adcData.reshape(numRX, -1, numADCSamples)
    print('Shape of radar data:', adcDataReshape.shape)
    return adcDataReshape



In [38]:
horiData = process_data(hori_bin, numLanes, numRX, numADCSamples, adcRatio)
vertData = process_data(vert_bin, numLanes, numRX, numADCSamples, adcRatio)

for idxFrame in range (0, numFrame):
    frameHori = horiData[:, numChirp*(idxFrame):numChirp*(idxFrame+1), 0:numADCSamples]
    frameVert = vertData[:, numChirp*(idxFrame):numChirp*(idxFrame+1), 0:numADCSamples]
    # outputHori = self.generateHeatmap(frameHori)
    # outputVert = self.generateHeatmap(frameVert)
    # self.saveRadarData(outputHori, self.saveDirNameGroup[idxName] + '/hori', idxFrame)
    # self.saveRadarData(outputVert, self.saveDirNameGroup[idxName] + '/vert', idxFrame)
    # print('%s, finished frame %d' % (self.radarDataFileNameGroup[idxName][0], idxFrame), end='\r')

Shape of radar data: (4, 115200, 256)
Shape of radar data: (4, 115200, 256)


In [39]:
print(horiData)

[[[ -71. -60.j  236. -22.j -135.-144.j ...   29. -25.j   33. -21.j
    -70.+116.j]
  [-187.-227.j   94. -29.j  -44. -34.j ... -107. +42.j  101. -60.j
    104.+149.j]
  [ 155. -26.j  184.-234.j  -36.+196.j ... -147. -43.j  138. -64.j
     92. -19.j]
  ...
  [ -96.+190.j   51.-296.j -166.-199.j ...   43. -23.j  -64.+114.j
   -102.  +6.j]
  [-308. +13.j -101. -79.j  -12.-264.j ...   56.+101.j  179. +81.j
    -75.+233.j]
  [  63.+222.j   95.-353.j  -61. +55.j ...   30. +60.j   90. +26.j
    -77. -36.j]]

 [[-175. +43.j  146. +93.j -150.  +2.j ...  192. -48.j  166. -44.j
    135. +43.j]
  [  33. +89.j  135.+192.j  211.+272.j ...  -76.  -5.j   18.-263.j
     13. -90.j]
  [-219.+107.j -135.-175.j -289.+234.j ... -155. -77.j  -25.-139.j
     31. +25.j]
  ...
  [-181.+202.j -173. -21.j -125. -68.j ...  195.  -1.j  203. +96.j
      7. -89.j]
  [ -20.+433.j   37.+197.j  120.  +1.j ...  153. -44.j   57.-226.j
   -107. +77.j]
  [ -99.+305.j -370.-248.j -348.+230.j ...  -33.-114.j -114. +12.j
    -3

In [40]:
print(vertData)

[[[ -47.-210.j  166.-148.j  263. +41.j ...   -6. +80.j   -9. +35.j
   -107. +18.j]
  [-145. -12.j  -56.-165.j   86.-109.j ...  -22. +93.j   52.+148.j
    -35. +37.j]
  [ 116.+202.j -110. +12.j   10.  -3.j ...  -58. +35.j   11. -85.j
   -113. +46.j]
  ...
  [  75. +26.j   49.-136.j   84.-116.j ...   -2. +62.j   17. -18.j
   -101. +27.j]
  [ -24. +48.j  -53. -74.j  -89. -72.j ...  -38.  -1.j   88.+233.j
     -7. +25.j]
  [ 103.+250.j   15. +24.j -123. +25.j ... -143. -66.j   66. +71.j
    -56. -51.j]]

 [[-281. +24.j  -59.-104.j  -73. -77.j ...  100. +39.j   71.+171.j
      2.+136.j]
  [ -26.+176.j -205.+127.j  -15. +58.j ...   -1.  -4.j  129.-120.j
    -59. -22.j]
  [ -35. -41.j -135.+147.j -228.-185.j ...   99. -76.j  -29. -16.j
     42. +97.j]
  ...
  [-168.+222.j -233. -53.j -300.-191.j ...   61. +76.j   40.+129.j
     31. +76.j]
  [ -49.+168.j  -80. +65.j  -19.+237.j ...  -61. -44.j  182. -63.j
    -81.-163.j]
  [ -44.-112.j   75.+102.j -300. -16.j ...    5. -56.j   23. -16.j
     1